In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-ddc5e13ad265>", line 6, in <cell line: 6>
    import tensorflow as tf
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtrace

In [22]:
def extract_Total_RE(eskom_dataframe, dictOfDates):
    for index, row in eskom_dataframe.iterrows():
        # Accessing values
        column1_value = row['Date']
        date_obj = datetime.strptime(column1_value, '%A, %d %B %Y')
        formatted_date = date_obj.strftime('%Y/%m/%d')
        if formatted_date in dictOfDates:
            tRE = dictOfDates[formatted_date]
            temp = tRE + row['Total_RE']
            dictOfDates[formatted_date] = temp
    return dictOfDates


def analyize_Weather(weather_df):
    DictionaryOfWeather = {}
    for index, row in weather_df.iterrows():
        DateTime = row['datetime']
        date_obj = datetime.strptime(DateTime, '%Y/%m/%d')

        formatted_date = date_obj.strftime('%Y/%m/%d')
        DictionaryOfWeather[formatted_date] = 0

    return DictionaryOfWeather

def preprocess_weather(weather_df, dictionaryOfDates, theTotalRE):
    #Here I will remove all the unnecassiry columns
    #And binary encode certain values
    weather_df.drop('conditions', axis=1, inplace=True)
    weather_df.drop('description', axis=1, inplace=True)
    weather_df.drop('stations', axis=1, inplace=True)
    weather_df.drop('sunrise', axis=1, inplace=True)
    weather_df.drop('sunset', axis=1, inplace=True)
    weather_df.drop('preciptype', axis=1, inplace=True)

    columns_to_drop = [col for col in weather_df.columns if 'Unnamed' in col]

    # Dropping the columns
    weather_df.drop(columns=columns_to_drop, inplace=True)
    icon_mapping = {
        'clear-day': 0,
        'rain': 1,
        'wind': 2,
        'partly-cloudy-day': 3,
        'cloudy': 4
    }
    weather_df['icon'] = weather_df['icon'].replace(icon_mapping)
    return average_locations(weather_df,dictionaryOfDates,theTotalRE)
def average_locations(weather_df, dictionaryOfDates,theTotalRE):
    finalVersion = pd.DataFrame()
    for x in dictionaryOfDates:
        results = weather_df.loc[weather_df["datetime"] == x]
        # Avoid SettingWithCopyWarning by creating a new DataFrame directly
        results = results.drop('name', axis=1)
        results = results.drop('datetime', axis=1)
        # Select only numerical columns for calculating the mean
        # Select only numerical columns except 'icon' for calculating the mean
        numerical_cols = results.select_dtypes(include=['number'])

        # Calculating the average for each numerical field except 'icon'
        averages = numerical_cols.mean()

        # Calculating the median for 'icon'
        icon_median = results['icon'].mode()

        # Creating a new DataFrame with the averages
        average_df = averages.to_frame().transpose()

        # Adding the median of 'icon' to the DataFrame
        average_df['icon'] = icon_median
        average_df['Total_RE'] = theTotalRE[x]
        finalVersion = pd.concat([finalVersion, average_df], ignore_index=True)
    finalVersion = finalVersion.dropna(subset=['Total_RE'])

    for column in finalVersion.columns:
      if finalVersion[column].isna().any():  # Check if there are any NaN values in the column
          mean_value = finalVersion[column].mean()  # Calculate the mean of the column
          finalVersion[column].fillna(mean_value, inplace=True)  # Replace NaNs with the mean value
    return finalVersion

def readInputFile(pathForWeather, pathForEskom):
    dfweather = pd.read_csv(pathForWeather)
    dfeskomData = pd.read_csv(pathForEskom)
    dict_of_dates = analyize_Weather(dfweather)
    theTotalRE = extract_Total_RE(dfeskomData,dict_of_dates)
    preprossed_df = preprocess_weather(dfweather,dict_of_dates,theTotalRE)
    return preprossed_df

preprossed_df = readInputFile('/content/drive/MyDrive/SACAIR-Hackathon-2023-TheWeather/Data/WeatherData.csv', '/content/drive/MyDrive/SACAIR-Hackathon-2023-TheWeather/Data/EskomData.csv')
print(preprossed_df)
normalized_df = (preprossed_df - preprossed_df.min()) / (preprossed_df.max() - preprossed_df.min())
preprossed_df = normalized_df

     tempmax  tempmin     temp  feelslikemax  feelslikemin  feelslike  \
0    30.2875  16.7500  23.1125       29.4625       16.7500    22.8500   
1    29.1500  16.3875  22.7625       28.9375       16.3875    22.6750   
2    29.5250  17.0250  22.8125       29.6375       17.0250    22.7875   
3    27.7500  16.8500  21.8625       27.7875       16.8500    21.8500   
4    23.8250  15.8125  19.4875       23.5750       15.8125    19.4500   
..       ...      ...      ...           ...           ...        ...   
646  30.2625  14.5375  22.3250       29.1625       14.5375    21.9625   
647  28.8000  16.8625  22.4125       28.3625       16.8625    22.1875   
648  27.1750  15.5750  21.2625       26.2000       15.5750    20.9875   
649  27.8250  13.5625  20.8500       26.8375       13.5625    20.5750   
650  27.6125  15.3750  21.4500       26.3250       15.3750    21.0000   

         dew  humidity    precip  precipprob  ...  sealevelpressure  \
0    13.4750   60.8750  0.021500        25.0  ...   

In [25]:
normalized_df = (preprossed_df - preprossed_df.min()) / (preprossed_df.max() - preprossed_df.min())
normalized_df.to_csv('/content/drive/MyDrive/SACAIR-Hackathon-2023-TheWeather/Data/processed.csv',index=False)